## In this notebook, I want to take our survey flux densities (at 1.4 or 3 GHz) and errors, and:

* Compute 6 GHz luminosities and their corresponding errors by propagating through the measured errors on their flux density from imfit (both for the case of $\alpha=0.0$ and $\alpha=-0.7$


* Save the 6 GHz flux density and corrseponding error that luminosity corresponds to. These may be useful for later computing variability amplitudes (Barvainis et al. 2005)

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import richardsplot
from astropy.cosmology import FlatLambdaCDM

In [45]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

def radluminosity(nu1, nu2, S_nu2, alpha, z):
    #calculate luminosity at nu1 based on data at nu2
    #input flux density units are µJy
    DL = 3.086e24*cosmo.luminosity_distance(z).value
    L = ((4*np.pi*DL**2*(S_nu2*1e-23*1e-6)) / ((1+z)**(1+alpha))) * (nu1/nu2)**alpha * 1e-7
    return np.log10(L)

def LtoS(nu1, nu2, L_nu2, alpha, z):
    #Compute flux at nu2 given log L at nu1
    DL = 3.086e24*cosmo.luminosity_distance(z).value
    L = 10**L_nu2
    S = L * (((1+z)**(1+alpha))/(4*np.pi*DL**2)) * 1e36 * (nu2/nu1)**alpha
    return S
    
def isNAN(arr):
    #this isnan works for strings too; np.isnan(a) only works if all elements
    #is arr are floats
    return arr!=arr

In [46]:
df = pd.read_csv("all_var_figs.csv")
df

,Name,z,S1p4_FIRST,S1p4_err,L6_FIRST,L6_FIRST_alpha0,DATE_FIRST,S3_VLASS,S3_quaderr,L6_VLASS,...,DATE_VLASS,S(6)_p,pmp,DATE_VLAC,S6_Aconfig,S6_Aconfig_err,DATE_VLAA,CLASS,S3_VLASS_final,S3_quaderr_final
0,075403.60+481428.0,0.276,7910,160,23.80,24.17,1997-04-10,4070.0,820.0,23.74,...,2019-05-10,2631.0,84.8,2011-1-15,NaN,NaN,NaN,RI,4395.6,952.860785
1,083658.90+442602.2,0.254,9150,120,23.78,24.15,1997-02-28,3940.0,805.0,23.65,...,2019-05-24,2596.0,79.0,2011-1-15,NaN,NaN,NaN,RI,4255.2,933.669863
2,091702.11+212337.5,0.202,4190,130,23.23,23.61,1998-09-15,4190.0,850.0,23.46,...,2019-04-22,4790.0,144.4,2011-1-15,NaN,NaN,NaN,RI,4525.2,986.802708
3,094215.12+090015.8,0.213,1690,130,22.88,23.27,2000-01-15,1300.0,279.0,23.00,...,2017-10-02,1162.0,38.4,2011-1-15,NaN,NaN,NaN,RI,1495.0,342.416242
4,094603.94+013923.6,0.220,7330,150,23.55,23.93,1998-07-24,3810.0,773.0,23.50,...,2018-01-02,2471.0,76.1,2011-1-15,NaN,NaN,NaN,RI,4381.5,955.560764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,145824.46+363119.5,0.246,495,0,22.49,22.86,1994-07-17,NaN,NaN,22.67,...,2017-11-10,167.0,9.4,2010-12-25,207.0,31.000000,2019-09-20,RQ,NaN,NaN
66,084313.41+535718.8,0.218,516,150,22.39,22.77,1997-05-07,NaN,NaN,22.45,...,2017-09-26,410.0,14.7,2010-12-26,349.0,31.000000,2019-08-13,RQ,NaN,NaN
67,123532.83+410445.1,0.212,510,0,22.35,22.74,1994-09-02,NaN,NaN,22.50,...,2019-04-15,266.0,16.1,2010-12-27,554.0,67.000000,2019-08-13,RQ,NaN,NaN
68,161723.67+085414.7,0.206,450,0,22.28,22.66,2000-01-15,NaN,NaN,22.59,...,2019-03-18,381.0,15.2,2010-12-28,386.7,18.164801,2019-08-20,RQ,NaN,NaN


Save FIRST luminosities and luminosity errors first

In [47]:
z = df[["z"]].values.flatten()
S1p4_first = df[["S1p4_FIRST"]].values.flatten()
S1p4err = df[["S1p4_err"]].values.flatten()
S6_firstup_err = np.zeros(len(S1p4err))

In [49]:
L6_first = radluminosity(6., 1.4, S1p4_first, -0.7, z)
L6_firstup = radluminosity(6., 1.4, S1p4_first, 0.0, z)

#for errors, consider detections separately; errors change logarithmically
fracerr = S1p4err/S1p4_first #turn fractional uncertainty into a variable
L6_first_uplim = np.log10(1+fracerr)
L6_first_lolim = abs(np.log10(1-fracerr))

#Now do fluxes
S6_first = LtoS(6., 6., L6_first, -0.7, z)
S6_first_err = S6_first*fracerr
S6_firstup = LtoS(6., 6., L6_firstup, 0.0, z)    
S6_firstup_err = S6_firstup*fracerr
        

In [50]:
#Save new values to table
df["L6_FIRST"] = L6_first
df["L6_FIRST_alpha0"] = L6_firstup
df["L6_FIRST_uplim"] = L6_first_uplim
df["L6_FIRST_lolim"] = L6_first_lolim
df["S6_FIRST"] = S6_first
df["S6_FIRST_err"] = S6_first_err
df["S6_FIRST_alpha0"] = S6_firstup
df["S6_FIRST_alpha0_err"] = S6_firstup_err
df

,Name,z,S1p4_FIRST,S1p4_err,L6_FIRST,L6_FIRST_alpha0,DATE_FIRST,S3_VLASS,S3_quaderr,L6_VLASS,...,DATE_VLAA,CLASS,S3_VLASS_final,S3_quaderr_final,L6_FIRST_uplim,L6_FIRST_lolim,S6_FIRST,S6_FIRST_err,S6_FIRST_alpha0,S6_FIRST_alpha0_err
0,075403.60+481428.0,0.276,7910,160,23.800933,24.169254,1997-04-10,4070.0,820.0,23.74,...,NaN,RI,4395.6,952.860785,0.008697,0.008875,2856.013350,57.770182,7910.0,160.0
1,083658.90+442602.2,0.254,9150,120,23.784059,24.157667,1997-02-28,3940.0,805.0,23.65,...,NaN,RI,4255.2,933.669863,0.005659,0.005733,3303.732257,43.327636,9150.0,120.0
2,091702.11+212337.5,0.202,4190,130,23.225996,23.612479,1998-09-15,4190.0,850.0,23.46,...,NaN,RI,4525.2,986.802708,0.013270,0.013688,1512.856629,46.938273,4190.0,130.0
3,094215.12+090015.8,0.213,1690,130,22.882062,23.265776,2000-01-15,1300.0,279.0,23.00,...,NaN,RI,1495.0,342.416242,0.032185,0.034762,610.197543,46.938273,1690.0,130.0
4,094603.94+013923.6,0.220,7330,150,23.550088,23.932053,1998-07-24,3810.0,773.0,23.50,...,NaN,RI,4381.5,955.560764,0.008798,0.008980,2646.596442,54.159545,7330.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,145824.46+363119.5,0.246,495,0,22.486485,22.862038,1994-07-17,NaN,NaN,22.67,...,2019-09-20,RQ,NaN,NaN,0.000000,0.000000,178.726499,0.000000,495.0,0.0
66,084313.41+535718.8,0.218,516,150,22.388927,22.771390,1997-05-07,NaN,NaN,22.45,...,2019-08-13,RQ,NaN,NaN,0.110825,0.149169,186.308835,54.159545,516.0,150.0
67,123532.83+410445.1,0.212,510,0,22.357267,22.741231,1994-09-02,NaN,NaN,22.50,...,2019-08-13,RQ,NaN,NaN,0.000000,0.000000,184.142454,0.000000,510.0,0.0
68,161723.67+085414.7,0.206,450,0,22.275612,22.661085,2000-01-15,NaN,NaN,22.59,...,2019-08-20,RQ,NaN,NaN,0.000000,0.000000,162.478636,0.000000,450.0,0.0


Re-compute the 6 GHz Luminosities from 2011

In [52]:
L6_kell_p7 = radluminosity(6., 6., df[["S(6)_p"]].values.flatten(), -0.7, z)
L6_kell_0 = radluminosity(6., 6., df[["S(6)_p"]].values.flatten(), 0.0, z)

fracerr = df[["pmp"]].values.flatten()/df[["S(6)_p"]].values.flatten()
kell_uplim = np.log10(1+fracerr)
kell_lolim = abs(np.log10(1-fracerr))

df["L6_Kell"] = L6_kell_p7
df["L6_Kell_alpha0"] = L6_kell_0
df["L6_Kell_uplim"] = kell_uplim
df["L6_Kell_lolim"] = kell_lolim
df

,Name,z,S1p4_FIRST,S1p4_err,L6_FIRST,L6_FIRST_alpha0,DATE_FIRST,S3_VLASS,S3_quaderr,L6_VLASS,...,L6_FIRST_uplim,L6_FIRST_lolim,S6_FIRST,S6_FIRST_err,S6_FIRST_alpha0,S6_FIRST_alpha0_err,L6_Kell,L6_Kell_alpha0,L6_Kell_uplim,L6_Kell_lolim
0,075403.60+481428.0,0.276,7910,160,23.800933,24.169254,1997-04-10,4070.0,820.0,23.74,...,0.008697,0.008875,2856.013350,57.770182,7910.0,160.0,23.765294,23.691198,0.013777,0.014228
1,083658.90+442602.2,0.254,9150,120,23.784059,24.157667,1997-02-28,3940.0,805.0,23.65,...,0.005659,0.005733,3303.732257,43.327636,9150.0,120.0,23.679359,23.610551,0.013019,0.013421
2,091702.11+212337.5,0.202,4190,130,23.225996,23.612479,1998-09-15,4190.0,850.0,23.46,...,0.013270,0.013688,1512.856629,46.938273,4190.0,130.0,23.726534,23.670601,0.012899,0.013294
3,094215.12+090015.8,0.213,1690,130,22.882062,23.265776,2000-01-15,1300.0,279.0,23.00,...,0.032185,0.034762,610.197543,46.938273,1690.0,130.0,23.161798,23.103095,0.014120,0.014594
4,094603.94+013923.6,0.220,7330,150,23.550088,23.932053,1998-07-24,3810.0,773.0,23.50,...,0.008798,0.008980,2646.596442,54.159545,7330.0,150.0,23.520273,23.459822,0.013173,0.013585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,145824.46+363119.5,0.246,495,0,22.486485,22.862038,1994-07-17,NaN,NaN,22.67,...,0.000000,0.000000,178.726499,0.000000,495.0,0.0,22.457012,22.390149,0.023782,0.025160
66,084313.41+535718.8,0.218,516,150,22.388927,22.771390,1997-05-07,NaN,NaN,22.45,...,0.110825,0.149169,186.308835,54.159545,516.0,150.0,22.731477,22.671524,0.015298,0.015857
67,123532.83+410445.1,0.212,510,0,22.357267,22.741231,1994-09-02,NaN,NaN,22.50,...,0.000000,0.000000,184.142454,0.000000,510.0,0.0,22.516994,22.458543,0.025521,0.027115
68,161723.67+085414.7,0.206,450,0,22.275612,22.661085,2000-01-15,NaN,NaN,22.59,...,0.000000,0.000000,162.478636,0.000000,450.0,0.0,22.645740,22.588797,0.016989,0.017681


Do the VLASS fluxes now

In [63]:
S3_vlass = df[["S3_VLASS_final"]].values.flatten()
S3err = df[["S3_quaderr_final"]].values.flatten()

vlass = ~isNAN(S3_vlass)

L6_vlass = np.zeros(len(S3err))
L6_vlass_uplim = np.zeros(len(S3err))
L6_vlass_lolim = np.zeros(len(S3err))

L6_vlassup = np.zeros(len(S3err)) #luminosity assuming alpha=0.0

S6_vlass = np.zeros(len(S3err))
S6_vlass_err = np.zeros(len(S3err))
S6_vlassup = np.zeros(len(S3err))
S6_vlassup_err = np.zeros(len(S3err))

In [64]:
L6_vlass[vlass] = radluminosity(6., 3., S3_vlass[vlass], -0.7, z[vlass])
L6_vlassup[vlass] = radluminosity(6., 3., S3_vlass[vlass], 0.0, z[vlass])

fracerr = S3err[vlass]/S3_vlass[vlass]
L6_vlass_uplim[vlass] = np.log10(1+fracerr)
L6_vlass_lolim[vlass] = abs(np.log10(1-fracerr))

S6_vlass[vlass] = LtoS(6., 6., L6_vlass[vlass], -0.7, z[vlass])
S6_vlass_err[vlass] = S6_vlass[vlass]*fracerr
S6_vlassup[vlass] = LtoS(6., 6., L6_vlassup[vlass], 0.0, z[vlass])
S6_vlassup_err[vlass] = S6_vlassup[vlass]*fracerr

In [65]:
#Save new values to table
df["L6_VLASS"] = L6_vlass
df["L6_VLASS_alpha0"] = L6_vlassup
df["L6_VLASS_uplim"] = L6_vlass_uplim
df["L6_VLASS_lolim"] = L6_vlass_lolim
df["S6_VLASS"] = S6_vlass
df["S6_VLASS_err"] = S6_vlass_err
df["S6_VLASS_alpha0"] = S6_vlassup
df["S6_VLASS_alpha0_err"] = S6_vlassup_err
df

,Name,z,S1p4_FIRST,S1p4_err,L6_FIRST,L6_FIRST_alpha0,DATE_FIRST,S3_VLASS,S3_quaderr,L6_VLASS,...,L6_Kell,L6_Kell_alpha0,L6_Kell_uplim,L6_Kell_lolim,L6_VLASS_uplim,L6_VLASS_lolim,S6_VLASS,S6_VLASS_err,S6_VLASS_alpha0,S6_VLASS_alpha0_err
0,075403.60+481428.0,0.276,7910,160,23.800933,24.169254,1997-04-10,4070.0,820.0,23.777470,...,23.765294,23.691198,0.013777,0.014228,0.085211,0.106114,2705.809192,586.554616,4395.6,952.860785
1,083658.90+442602.2,0.254,9150,120,23.784059,24.157667,1997-02-28,3940.0,805.0,23.683253,...,23.679359,23.610551,0.013019,0.013421,0.086153,0.107582,2619.382854,574.741218,4255.2,933.669863
2,091702.11+212337.5,0.202,4190,130,23.225996,23.612479,1998-09-15,4190.0,850.0,23.491115,...,23.726534,23.670601,0.012899,0.013294,0.085672,0.106831,2785.587350,607.448320,4525.2,986.802708
3,094215.12+090015.8,0.213,1690,130,22.882062,23.265776,2000-01-15,1300.0,279.0,23.060512,...,23.161798,23.103095,0.014120,0.014594,0.089566,0.112969,920.280449,210.781921,1495.0,342.416242
4,094603.94+013923.6,0.220,7330,150,23.550088,23.932053,1998-07-24,3810.0,773.0,23.558302,...,23.520273,23.459822,0.013173,0.013585,0.085679,0.106843,2697.129624,588.216648,4381.5,955.560764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,145824.46+363119.5,0.246,495,0,22.486485,22.862038,1994-07-17,NaN,NaN,0.000000,...,22.457012,22.390149,0.023782,0.025160,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
66,084313.41+535718.8,0.218,516,150,22.388927,22.771390,1997-05-07,NaN,NaN,0.000000,...,22.731477,22.671524,0.015298,0.015857,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
67,123532.83+410445.1,0.212,510,0,22.357267,22.741231,1994-09-02,NaN,NaN,0.000000,...,22.516994,22.458543,0.025521,0.027115,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
68,161723.67+085414.7,0.206,450,0,22.275612,22.661085,2000-01-15,NaN,NaN,0.000000,...,22.645740,22.588797,0.016989,0.017681,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


A configuration luminosities re-computed

In [66]:
L6_Aconfig_p7 = np.zeros(len(df[["S6_Aconfig"]].values.flatten()))
L6_Aconfig_0 = np.zeros(len(df[["S6_Aconfig"]].values.flatten()))

Aconfig_uplim = np.zeros(len(df[["S6_Aconfig"]].values.flatten()))
Aconfig_lolim = np.zeros(len(df[["S6_Aconfig"]].values.flatten()))

L6_Aconfig_p7[~vlass] = radluminosity(6., 6., df[["S6_Aconfig"]].values.flatten()[~vlass], -0.7, z[~vlass])
L6_Aconfig_0[~vlass] = radluminosity(6., 6., df[["S6_Aconfig"]].values.flatten()[~vlass], 0.0, z[~vlass])

fracerr = df[["S6_Aconfig_err"]].values.flatten()[~vlass]/df[["S6_Aconfig"]].values.flatten()[~vlass]
Aconfig_uplim[~vlass] = np.log10(1+fracerr)
Aconfig_lolim[~vlass] = np.log10(1-fracerr)

In [67]:
df["L6_Aconfig"] = L6_Aconfig_p7
df["L6_Aconfig_alpha0"] = L6_Aconfig_0
df["L6_Aconfig_uplim"] = Aconfig_uplim
df["L6_Aconfig_lolim"] = Aconfig_lolim
df

,Name,z,S1p4_FIRST,S1p4_err,L6_FIRST,L6_FIRST_alpha0,DATE_FIRST,S3_VLASS,S3_quaderr,L6_VLASS,...,L6_VLASS_uplim,L6_VLASS_lolim,S6_VLASS,S6_VLASS_err,S6_VLASS_alpha0,S6_VLASS_alpha0_err,L6_Aconfig,L6_Aconfig_alpha0,L6_Aconfig_uplim,L6_Aconfig_lolim
0,075403.60+481428.0,0.276,7910,160,23.800933,24.169254,1997-04-10,4070.0,820.0,23.777470,...,0.085211,0.106114,2705.809192,586.554616,4395.6,952.860785,0.000000,0.000000,0.000000,0.000000
1,083658.90+442602.2,0.254,9150,120,23.784059,24.157667,1997-02-28,3940.0,805.0,23.683253,...,0.086153,0.107582,2619.382854,574.741218,4255.2,933.669863,0.000000,0.000000,0.000000,0.000000
2,091702.11+212337.5,0.202,4190,130,23.225996,23.612479,1998-09-15,4190.0,850.0,23.491115,...,0.085672,0.106831,2785.587350,607.448320,4525.2,986.802708,0.000000,0.000000,0.000000,0.000000
3,094215.12+090015.8,0.213,1690,130,22.882062,23.265776,2000-01-15,1300.0,279.0,23.060512,...,0.089566,0.112969,920.280449,210.781921,1495.0,342.416242,0.000000,0.000000,0.000000,0.000000
4,094603.94+013923.6,0.220,7330,150,23.550088,23.932053,1998-07-24,3810.0,773.0,23.558302,...,0.085679,0.106843,2697.129624,588.216648,4381.5,955.560764,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,145824.46+363119.5,0.246,495,0,22.486485,22.862038,1994-07-17,NaN,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,22.550266,22.483403,0.060607,-0.070458
66,084313.41+535718.8,0.218,516,150,22.388927,22.771390,1997-05-07,NaN,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,22.661519,22.601566,0.036958,-0.040398
67,123532.83+410445.1,0.212,510,0,22.357267,22.741231,1994-09-02,NaN,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,22.835623,22.777171,0.049582,-0.055981
68,161723.67+085414.7,0.206,450,0,22.275612,22.661085,2000-01-15,NaN,NaN,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,22.652190,22.595246,0.019936,-0.020895


In [68]:
df.to_csv("all_var_figs.csv", index=False)

Done 9/1/2020